# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Рассмотрите, как можно выстраивать конвейер предобработки данных при помощи `Pipeline` из `sklearn`

2. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class SyntDataset(Dataset):
  def __init__(self, transform: callable = None, **make_regression_args):
    self.X, self.y = make_regression(**make_regression_args)
    self.transform = transform

  def __getitem__(self, idx):
    x = self.X[idx]
    if self.transform is not None:
      x = self.transform(x)

    return x, self.y[idx]

  def __len__(self):
    return len(self.X)

In [ ]:
import numpy as np

def add_squares(x):
  return np.c_[x, x**2]

In [ ]:
d = SyntDataset(transform=add_squares, n_samples=1000, n_features=5)
d[0:5]

In [ ]:
class SquareN:
  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    for _ in range(self.n):
      x = add_squares(x)
    return x

In [ ]:
# o = SquareN(2)
# o(d.X[:4])
# add_squares(d.X[:4])
d = SyntDataset(
    transform=SquareN(2),
    n_samples=1000,
    n_features=5
)

In [ ]:
dl = DataLoader(d, batch_size=32)

In [ ]:
for (batch_X, batch_y) in dl:
  print(batch_X.shape, batch_y.shape)
  break

torch.Size([32, 5, 4]) torch.Size([32])


## Задачи для самостоятельного решения

In [114]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from torch.utils.data import Dataset, DataLoader

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`. Используя `Pipeline` из `sklearn`, закодируйте значения в нечисловых столбцах целыми числами, после чего нормализуйте получившиеся признаки. Выведите преобразованные данные на экран.

In [5]:
df = pd.read_csv('bank-full.csv')
df.head(2)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no


In [6]:
cols_obj = list(df.columns.to_series().groupby(df.dtypes).groups.items())[1][1].values
cols_obj

array(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'poutcome', 'y'], dtype=object)

In [7]:
cols_num = list(df.columns.to_series().groupby(df.dtypes).groups.items())[0][1].values
cols_num

array(['age', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous'], dtype=object)

In [253]:
ct = ColumnTransformer(
    [
        ('encoding', OrdinalEncoder(), cols_obj),
        ('scaling', StandardScaler(), cols_num)
    ]
)

In [254]:
pipe = Pipeline(
    [
        ('transformer', ct)
    ]
).fit_transform(df)

In [255]:
pipe

array([[ 4.        ,  1.        ,  2.        , ..., -0.56935064,
        -0.41145311, -0.25194037],
       [ 9.        ,  2.        ,  1.        , ..., -0.56935064,
        -0.41145311, -0.25194037],
       [ 2.        ,  1.        ,  1.        , ..., -0.56935064,
        -0.41145311, -0.25194037],
       ...,
       [ 5.        ,  1.        ,  1.        , ...,  0.72181052,
         1.43618859,  1.05047333],
       [ 1.        ,  1.        ,  1.        , ...,  0.39902023,
        -0.41145311, -0.25194037],
       [ 2.        ,  1.        ,  1.        , ..., -0.24656035,
         1.4761376 ,  4.52357654]])

<p class="task" id="2"></p>

2\. Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` и `i`-ую строку из `obj.y`.
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

In [251]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        self.X, self.y = data.drop(columns='y').values, data['y'].values

    def __getitem__(self, idx) -> tuple:
        return (self.X[idx], self.y[idx])

    def __len__(self) -> int:
        return len(self.X)

In [252]:
BD = BankDatasetBase(df)
BD[0:2]

(array([[58, 'management', 'married', 'tertiary', 'no', 2143, 'yes', 'no',
         'unknown', 5, 'may', 261, 1, -1, 0, 'unknown'],
        [44, 'technician', 'single', 'secondary', 'no', 29, 'yes', 'no',
         'unknown', 5, 'may', 151, 1, -1, 0, 'unknown']], dtype=object),
 array(['no', 'no'], dtype=object))

<p class="task" id="3"></p>

3\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательный аргумент `transform: callable`;
* если данный аргумент был передан, то при получении `i`-го элемента, нужно вызвать `transform(x, y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность.

In [8]:
class BankDataset(Dataset):
    def __init__(self, data: pd.DataFrame, transform: callable = None) -> None:
        self.X, self.y = data.drop(columns='y'), data['y']
        self.transform = transform

    def __getitem__(self, idx) -> tuple:
        # x - набор признаков из idx-й строки
        # y - набор признаков из idx-й строки
        # если при создании был передан transform
        # x, y = transform(x, y)
        x = self.X.iloc[idx]
        y = self.y[idx]
        if self.transform is not None:
          x, y = self.transform(x, y)
        return x, y

    def __len__(self) -> int:
        return len(self.X)

In [9]:
import numpy as np

In [10]:
def trans(x, y):
  x = x.values
  x_num = x[0]
  return np.r_[x, x_num**2], y

In [11]:
BD = BankDataset(df, trans)
BD[2]

(array([33, 'entrepreneur', 'married', 'secondary', 'no', 2, 'yes', 'yes',
        'unknown', 5, 'may', 76, 1, -1, 0, 'unknown', 1089], dtype=object),
 'no')

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет два параметра (признаки и класс) и возвращает кортеж, состоящих двух элементов: набора признаков, в котором нечисловые характеристики закодированы целыми числами; и чисел - меток классов;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `LabelEncoderTransform`.

In [169]:
class LabelEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.cols = category_columns
        self.encoders = {col: {} for col in self.cols}

    def encoding(self, X, column):
        if X not in self.encoders[column].keys():
          try:
            self.encoders[column][X] = max(self.encoders[column].values()) + 1
          except ValueError:
            self.encoders[column][X] = 0
        return self.encoders[column][X]

    def __call__(self, x, y) -> tuple:
        target_col = 'y'
        feature_cols = set(self.cols) - set(target_col)
        encoded = x.copy()
        labels = []
        for col in feature_cols:
          if type(x) == pd.core.series.Series:
            encoded[col] = self.encoding(x[col], col)
          else:
            encoded[col] = encoded[col].apply(lambda el: self.encoding(el, col))

        if type(y) == str:
          labels = np.array([self.encoding(y, target_col)], dtype=np.float64)
        else:
          labels = (y.apply(lambda el: self.encoding(el, target_col))).values

        encoded = (encoded.values).astype(np.float64)
        return encoded, labels

In [167]:
BD = BankDataset(df, LabelEncoderTransform(cols_obj))

In [168]:
BD[3]

(array([ 4.700e+01,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         1.506e+03,  0.000e+00,  0.000e+00,  0.000e+00,  5.000e+00,
         0.000e+00,  9.200e+01,  1.000e+00, -1.000e+00,  0.000e+00,
         0.000e+00]),
 array([0.]))

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и возвращает кортеж, состоящих 2 тензоров.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании список из объектов LabelEncoderTransform и ToTensor.

In [104]:
import torch as th

In [105]:
class ToTensor(object):
    def __call__(self, X, y) -> tuple:
        return th.Tensor(X), th.Tensor(y)

class Compose(object):
    def __init__(self, transforms: list) -> None:
        self.transforms = transforms

    def __call__(self, X, y):
        for transformer in self.transforms:
          X, y = transformer(X, y)
        return X, y

In [170]:
transformers = [
    LabelEncoderTransform(cols_obj),
    ToTensor()
]
BD = BankDataset(df, Compose(transformers))

In [230]:
BD[:3]

(tensor([[ 5.8000e+01,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           2.1430e+03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
           0.0000e+00,  2.6100e+02,  1.0000e+00, -1.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 4.4000e+01,  2.0000e+00,  1.0000e+00,  2.0000e+00,  0.0000e+00,
           2.9000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
           0.0000e+00,  1.5100e+02,  1.0000e+00, -1.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 3.3000e+01,  3.0000e+00,  0.0000e+00,  2.0000e+00,  0.0000e+00,
           2.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  5.0000e+00,
           0.0000e+00,  7.6000e+01,  1.0000e+00, -1.0000e+00,  0.0000e+00,
           0.0000e+00]]),
 tensor([0., 0., 0.]))

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

In [185]:
from torch.utils.data import random_split, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [174]:
train, test = random_split(BD, [0.75, 0.25])

In [250]:
def collate_fn_cust(data: tuple[th.Tensor, th.Tensor]):
    data, targets = zip(*data)
    data = th.stack(data).reshape((64, 2, 8))
    targets = th.Tensor(targets)
    return data, targets

dl = DataLoader(train, batch_size=64, shuffle=True, collate_fn=collate_fn_cust)
for batch_X, batch_y in dl:
  print(batch_X.shape, batch_y.shape)
  break

torch.Size([64, 2, 8]) torch.Size([64])


## Обратная связь
- [x] Хочу получить обратную связь по решению